In [2]:
import numpy as np

## Example 6.1 from Kersting book

In [284]:
Z_abc =  np.array([
[0.8667 + 2.0417j, 0.2955 + 0.9502j, 0.2907 + 0.7290j],
[0.2955 + 0.9502j, 0.8837 + 1.9852j, 0.2992 + 0.8023j],
[0.2907 + 0.7290j, 0.2992 + 0.8023j, 0.8741 + 2.0172j]
]) # Ohm

Y_abc =  np.array([
[ 10.7409j, -3.4777j, -1.3322j],
[ -3.4777j, 11.3208j, -2.2140j],
[ -1.3322j, -2.2140j, 10.2104j]
])*1.0e-6 # S

### Incidence matrix

In [285]:
A = np.zeros(([6,9]),dtype=np.float64)
A[0,0] =  1
A[3,0] = -1
A[1,1] =  1
A[4,1] = -1
A[2,2] =  1
A[5,2] = -1
A[0,3] =  1
A[1,4] =  1
A[2,5] =  1
A[3,6] =  1
A[4,7] =  1
A[5,8] =  1

In [286]:
def diag_2d(Y_prim_list):

    N_cols = 0

    for Y_prim in Y_prim_list:
        N_cols += Y_prim.shape[1]

    Y_prims = np.zeros((N_cols,N_cols),dtype=np.complex128)

    it = 0
    for Y_prim in Y_prim_list:
        N = Y_prim.shape[0] 
        Y_prims[it:(it+N),it:(it+N)] = Y_prim
        it += N

    return Y_prims

### Primitive matrix

In [287]:
Y_prim_list = [np.linalg.inv(Z_abc),Y_abc/2,Y_abc/2]
Y_prim = diag_2d(Y_prim_list)

### Bus matrix

In [288]:
Y_bus = A @ Y_prim @ A.T

### Solution

In [290]:
V_unknown = np.linalg.solve(Y_ii, I_known - Y_iv @ V_known)

Y_ii = Y_bus[3:6,3:6]
Y_iv = Y_bus[0:3,3:6]
phi = -np.arccos(0.9)
I_known = -277.79*np.array([1,alpha**2,alpha]).reshape(3,1)*np.exp(1j*phi)
V_known = 12470/np.sqrt(3)*np.array([1,alpha**2,alpha]).reshape(3,1)
V_known_end = np.copy(V_known)

for it in range(10):
    V_unknown = np.linalg.solve(Y_ii, I_known - Y_iv @ V_known)
    DU = V_known - V_unknown
    V_known = V_known_end + DU

Phases = ['a','b','c']
for ph,v in zip(Phases,V_known[:,0]):
    print('V_{:s} = {:2.5f}|{:2.5f}º kV'.format(ph,abs(v)/1000,np.angle(v,deg=True)))
V_avg = np.sum(abs(V_known))/3
print('V_avg = {:2.2f} V'.format(V_avg))
U = V_avg*np.sqrt(3)
print('U = {:2.4f} kV'.format(U/1000))

V_a = 7.53869|1.57248º kV
V_b = 7.45125|-118.30047º kV
V_c = 7.48512|121.93184º kV
V_avg = 7491.68 V
U = 12.9760 kV


## Transposed case

In [233]:
def transpose(Z_abc):
    '''
    Kersting pg. 91
    '''
    Z_abc_transpose = np.zeros((3,3),dtype=np.complex128)
    Z_s = np.average(Z_abc.diagonal())
    Z_abc_transpose[0,0] = Z_s
    Z_abc_transpose[1,1] = Z_s    
    Z_abc_transpose[2,2] = Z_s
    
    N_off_diagonal = 0
    Z_m_sum = 0.0+0j
    for irow in range(3):
        for icol in range(3):
            if irow!=icol:
                Z_m_sum += Z_abc[irow,icol]
                N_off_diagonal += 1
                
    Z_m = Z_m_sum/N_off_diagonal

    for irow in range(3):
        for icol in range(3):
            if irow!=icol:
                Z_abc_transpose[irow,icol] = Z_m
                
    return Z_abc_transpose

In [243]:
Z_abc_transposed = transpose(Z_abc)
Y_abc_transposed = transpose(Y_abc)

### Primitive matrix

In [244]:
Y_prim_list = [np.linalg.inv(Z_abc_transposed),Y_abc_transposed/2,Y_abc_transposed/2]
Y_prim_transposed = diag_2d(Y_prim_list)

### Bus matrix

In [245]:
Y_bus_transposed = A @ Y_prim_transposed @ A.T

### Solution

In [266]:
Y_ii = Y_bus_transpose[3:6,3:6]
Y_iv = Y_bus_transpose[0:3,3:6]
phi = -np.arccos(0.9)
I_known = -277.79*np.array([1,alpha**2,alpha]).reshape(3,1)*np.exp(1j*phi)
V_known = 12470/np.sqrt(3)*np.array([1,alpha**2,alpha]).reshape(3,1)
V_known_end = np.copy(V_known)

for it in range(10):
    V_unknown = np.linalg.solve(Y_ii, I_known - Y_iv @ V_known)
    DU = V_known - V_unknown
    V_known = V_known_end + DU

Phases = ['a','b','c']
for ph,v in zip(Phases,V_known[:,0]):
    print('V_{:s}_n = {:2.2f}|{:2.2f}º V'.format(ph,abs(v),np.angle(v,deg=True)))

for ph,v in zip(Phases,V_unknown[:,0]):
    print('V_{:s}_m = {:2.2f}|{:2.2f}º V'.format(ph,abs(v),np.angle(v,deg=True)))


V_a_n = 7491.66|1.73º V
V_b_n = 7491.66|-118.27º V
V_c_n = 7491.66|121.73º V
V_a_m = 7199.56|0.00º V
V_b_m = 7199.56|-120.00º V
V_c_m = 7199.56|120.00º V


## Sequence impedances and solution in opendss

In [247]:
alpha = np.exp(1j*2/3*np.pi)
A_s = np.array([[1,    1,      1],
                [1,alpha**2,alpha],
                [1,alpha,alpha**2]])


In [257]:
Z_012 = np.linalg.inv(A_s) @ Z_abc_transposed @ A_s
Y_012 = np.linalg.inv(A_s) @ Y_abc_transposed @ A_s

In [268]:
R_0 = Z_012[0,0].real
R_1 = Z_012[1,1].real
X_0 = Z_012[0,0].imag
X_1 = Z_012[1,1].imag

Omega = 2.0*np.pi*60
B_0 = Y_012[0,0].imag
B_1 = Y_012[1,1].imag
C_0 = B_0/Omega
C_1 = B_1/Omega
C_0_nF = C_0*1e9
C_1_nF = C_1*1e9

In [269]:
C_1

3.4745292465217327e-08

In [270]:
string = ''
string += ('X1 = {:2.3f}  '.format(X_1))
string += ('X0 = {:2.3f}  '.format(X_0))
string += ('R1 = {:2.3f}  '.format(R_1))
string += ('R0 = {:2.3f}  '.format(R_0))
string += ('C1 = {:2.3f}  '.format(C_1_nF))
string += ('C0 = {:2.3f}  '.format(C_0_nF))

In [271]:
string

'X1 = 1.188  X0 = 3.669  R1 = 0.580  R0 = 1.465  C1 = 34.745  C0 = 16.114  '

In [282]:
print(Y_bus.real)

[[ 0.25691729 -0.12478409 -0.04504326 -0.25691729  0.12478409  0.04504326]
 [-0.12478409  0.28730429 -0.07646596  0.12478409 -0.28730429  0.07646596]
 [-0.04504326 -0.07646596  0.23085184  0.04504326  0.07646596 -0.23085184]
 [-0.25691729  0.12478409  0.04504326  0.25691729 -0.12478409 -0.04504326]
 [ 0.12478409 -0.28730429  0.07646596 -0.12478409  0.28730429 -0.07646596]
 [ 0.04504326  0.07646596 -0.23085184 -0.04504326 -0.07646596  0.23085184]]


In [283]:
print(Y_bus.imag)

[[-0.53272198  0.1828866   0.11878035  0.53272735 -0.18288834 -0.11878101]
 [ 0.1828866  -0.55205704  0.14497317 -0.18288834  0.5520627  -0.14497428]
 [ 0.11878035  0.14497317 -0.5141181  -0.11878101 -0.14497428  0.5141232 ]
 [ 0.53272735 -0.18288834 -0.11878101 -0.53272198  0.1828866   0.11878035]
 [-0.18288834  0.5520627  -0.14497428  0.1828866  -0.55205704  0.14497317]
 [-0.11878101 -0.14497428  0.5141232   0.11878035  0.14497317 -0.5141181 ]]


### OpenDSS

Yprim of active circuit element: Line.1-2


G matrix (conductance), S

 0.2525253908 
-0.07933099092  0.2525253908 
-0.07933099092 -0.07933099092  0.2525253908 
-0.2525253908 0.07933099092 0.07933099092  0.2525253908 
0.07933099092 -0.2525253908 0.07933099092 -0.07933099092  0.2525253908 
0.07933099092 0.07933099092 -0.2525253908 -0.07933099092 -0.07933099092  0.2525253908 

jB matrix (Susceptance), S

-0.5315085593 
 0.1482183078 -0.5315085593 
 0.1482183078  0.1482183078 -0.5315085593 
 0.5315139379 -0.1482194784 -0.1482194784 -0.5315085593 
-0.1482194784  0.5315139379 -0.1482194784  0.1482183078 -0.5315085593 
-0.1482194784 -0.1482194784  0.5315139379  0.1482183078  0.1482183078 -0.5315085593 


In [272]:
C_km = 20000000e-12 
Long = 100 # km
C = C_km*Long
Omega = 2.0*np.pi*50
X_c = 1.0/(Omega*C)

In [111]:
3*241.7**2/X_c/1000

110.11701339292212

In [98]:
X_c

1.5915494309189535

In opendss:
https://sourceforge.net/p/electricdss/discussion/beginners/thread/7096e0bc/

In [120]:
U = 20.0e3
C_km = 0.25e-6 
Long = 2 # km
C = C_km*Long
Omega = 2.0*np.pi*50
X_c = 1.0/(Omega*C)
Q = 3*(U/np.sqrt(3))**2/X_c
print('Q = {:2.3f} kvar'.format(Q/1000))

Q = 62.832 kvar


In [116]:
Q

62831.853071795857

In [121]:
R1 = 0.277
R0 = 0.831  
X1 = 0.118  
X0 =0.354   
C1=250    
C0=750  
Length=2.0 
units='km'  
baseFreq=50 

In [128]:
A

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [281]:
np.linalg.inv(Y_abc_transposed)

array([[ 0.-105767.55753017j,  0. -29423.9093217j ,  0. -29423.9093217j ],
       [ 0. -29423.9093217j ,  0.-105767.55753017j,  0. -29423.9093217j ],
       [ 0. -29423.9093217j ,  0. -29423.9093217j ,  0.-105767.55753017j]])

[[ 7538.69069037]
 [ 7451.24613542]
 [ 7485.11694409]]
[[ 7199.55785679]
 [ 7199.55785679]
 [ 7199.55785679]]
[[  1.02284455e-14]
 [ -1.20000000e+02]
 [  1.20000000e+02]]


In [213]:
np.abs(V_unknown)*np.sqrt(3)

array([[ 12470.],
       [ 12470.],
       [ 12470.]])

In [216]:
np.angle(-I_known,deg=True)

array([[ -25.84193276],
       [-145.84193276],
       [  94.15806724]])

In [146]:
I_known

array([[ 0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j]])